In [1]:
import numpy  as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib.pyplot as plt1
%matplotlib inline
import plotly.plotly as py
from plotly.graph_objs import*

C:\Users\Bibek\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


ModuleNotFoundError: No module named 'plotly'

In [ ]:
from statsmodels.graphics.api import qqplot

In [ ]:
data=pd.read_csv("Desktop\\Fake.csv")

In [ ]:
data.head()

In [ ]:
data.head(50)

In [ ]:
data.plot('Date',figsize=(15,5))

In [ ]:
data.info()

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
from datetime import datetime
con=data['Date']
data['Date']=pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
#check datatype of index
data.index

In [ ]:
my_plot=data.plot(kind='line',figsize=(15,5))
my_plot.set_xlabel("Date")
my_plot.set_ylabel("Fake")
my_plot.legend(["Fake"])


In [ ]:
ts=data['Fake']

In [ ]:
ts.head()

In [ ]:
ts['2016-03-20']

In [ ]:
plt.plot(ts)

In [ ]:
#Checking the stationery of time series


from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)
#Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
test_stationarity(ts)

In [ ]:
#This is not stationary because :

# mean is increasing even though the std is small.

# Test stat is > critical value.

#Note: the signed values are compared and the absolute values.

In [ ]:
ts_log=np.log(ts)
ts_log.plot(figsize=(15,5))

In [ ]:
ts_log.head(20)

In [ ]:
test_stationarity(ts_log)

In [ ]:
ts_log_diff = ts_log - ts_log.shift(1)
plt.plot(ts_log_diff)

In [ ]:
ts_log_diff.dropna(inplace=True)
test_stationarity(ts_log_diff)

In [ ]:
#df=data


In [ ]:
ts_log.head()

In [ ]:
#df.head()

In [ ]:
#df["FakeShift1"]=df.Fake.shift()

In [ ]:
#df.head()

In [ ]:
#df["Fakelag1"]=df.Fake-df.FakeShift1


In [ ]:
#df.head()

In [ ]:
#df.Fakelag1.plot(figsize=(15,5))

In [ ]:
#ts=df.Fakelag1

In [ ]:
#df.Fakelag1.dropna(inplace=True)
#test_stationarity(df.Fakelag1)

In [ ]:
#This is  stationary because :

# mean is increasing even though the std is small.

# Test stat is < critical value.

#Note: the signed values are compared and the absolute values.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#data.head()

In [ ]:
decomposition=seasonal_decompose(ts_log,model="additive")
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
fig = decomposition.plot()
fig.set_size_inches(15, 5)

In [ ]:
ts_log.head()

In [ ]:
ts_log_decompose = residual
ts_log_decompose.dropna(inplace=True)
test_stationarity(ts_log_decompose)

In [ ]:
ts_log_seasonal_first_difference=ts_log_diff-ts_log_diff.shift(12)
#ts['seasonal_first_difference']=df.Fakelag1-df.Fakelag1.shift(12)
test_stationarity(ts_log_seasonal_first_difference.dropna(inplace=False))

In [ ]:
data.head()

In [ ]:
df=ts_log
df_diff=ts_log_diff
df_diff.dropna(inplace=True)
df.head()

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
lag_acf = acf(df_diff, nlags=20)

In [ ]:
lag_acf

In [ ]:
#Plot Acf
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(df_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(df_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

In [ ]:
lag_pacf = pacf(df_diff, nlags=20, method='ols')


In [ ]:
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(df_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(df_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [ ]:

from statsmodels.tsa.arima_model import ARIMA

In [ ]:
ts_log.head()

In [ ]:
ts.head()

In [ ]:
model=ARIMA(df,order=(2, 1, 0))
results_AR=model.fit(display=-1)
results_AR.fittedvalues.head()

In [ ]:
#AR MODEL
plt.plot(ts_diff)
plt.plot(results_AR.fittedvalues,color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-df_diff)**2))


In [ ]:
model = ARIMA(df, order=(0, 1, 2))  
results_MA = model.fit(disp=-1)  
plt.plot(ts_diff)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-df_diff)**2))

In [ ]:
model = ARIMA(df, order=(2, 1, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-df_diff)**2))